## analyse exploratoire des contributions des votant·e·s à la question 1

In [1]:
import pandas as pd
import dask
dask.config.set({'dataframe.query-planning': True})
import dask.dataframe as dd
from tqdm import tqdm

/home/tk/.virtualenvs/kitchen/lib/python3.10/site-packages/dask/dataframe/_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 11.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(


In [2]:
votes = pd.read_csv('../data/votes-q1.csv')

#votes.columns

Index(['path', 'user', 'vote'], dtype='object')

In [3]:
contributions = (
    dd
    .read_csv('../data/user-logs/*.csv')
    .compute()
)

contributions.shape

(9742264, 16)

In [4]:
main = (
    contributions
    .query('~title.isna()')
    .query('~title.str.contains(":")')
)

main.shape

(6811318, 16)

In [5]:
main.columns

Index(['userid', 'user', 'pageid', 'revid', 'parentid', 'ns', 'title',
       'timestamp', 'comment', 'size', 'top', 'minor', 'new', 'commenthidden',
       'suppressed', 'texthidden'],
      dtype='object')

## tout le monde

In [6]:
(
    main
    .groupby('title')
    .agg({
        'userid': lambda s: len(s.unique())
    })
    .sort_values('userid', ascending=False)
    .head(10)
)

,userid
title,
Éric Zemmour,72
Charles de Gaulle,71
France,71
Jacques Chirac,66
Rassemblement national,66
Nicolas Sarkozy,64
Emmanuel Macron,64
François Mitterrand,63
François Hollande,60


In [7]:
oui = votes.query('vote == "oui"')
non = votes.query('vote == "non"')

## seulement `vote = oui`

In [8]:
top_oui = (
    main
    .query('user.isin(@oui.user)')
    .groupby('title')
    .agg({
        'userid': lambda s: len(s.unique())
    })
    .sort_values('userid', ascending=False)
)

In [10]:
(
    top_oui
    .head(10)
)

,userid
title,
France,42
Charles de Gaulle,39
Emmanuel Macron,39
Donald Trump,38
Éric Zemmour,37
Attentats du 11 septembre 2001,37
Jacques Chirac,36
François Mitterrand,36
Lyon,36


## seulement `vote = non`

In [9]:
top_non = (
    main
    .query('user.isin(@non.user)')
    .groupby('title')
    .agg({
        'userid': lambda s: len(s.unique())
    })
    .sort_values('userid', ascending=False)
)

In [11]:
(
    top_non
    .head(10)
)

,userid
title,
Philippe Pétain,18
Éric Zemmour,18
Marseille,16
Jean-Luc Mélenchon,16
Adolf Hitler,16
Charles de Gaulle,16
Twitter,16
Nantes,16
Nicolas Sarkozy,16
